
> **Binary classification model** : AntsBees  

> **Coding Stye**: TypeB

> **Section**: DataSet Class




In [1]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:

import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
import torch.utils.data as data
from pathlib import Path


In [3]:
data_dir = '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data'
root_dir= '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data'

In [4]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AntsBees')

# ***DataSet Class***

###（1）ImageTransform Class

In [5]:
from util.ImageTransform import ImageTransform

###(2) make_path_list function

### train_list, val_list,file_list 

In [6]:
from dsets.dsets import make_path_list

In [7]:
import pprint 
# ファイルパスのリストを生成
train_list = make_path_list(phase='train',root_dir=root_dir)
val_list = make_path_list(phase='val',root_dir=root_dir)

file_list={'train':train_list,'val':val_list}

#print(train_list)
print(len(train_list))

# 訓練データのファイルパスの前後5要素ずつ出力
print('train')
print(train_list[:2])
pprint.pprint(train_list[-2:-1])
# 検証データのファイルパスの前後5要素ずつ出力
print('val')
print(val_list[:2])
pprint.pprint(val_list[-3:-1])

242
train
[CandidateInfoTuple(label=1, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg'), CandidateInfoTuple(label=1, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/0013035.jpg')]
[CandidateInfoTuple(label=0, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/bees/2634617358_f32fd16bea.jpg')]
val
[CandidateInfoTuple(label=1, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/val/ants/10308379_1b6c72e180.jpg'), CandidateInfoTuple(label=1, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/val/ants/1053149811_f62a3410d3.jpg')]
[CandidateInfoTuple(label=0, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/val/bees/759745145_e8bc776ec8.jpg'),
 CandidateInfoTuple(label=0, path='/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/val/bees/936182217_c4c

###ここで、antsの画像ファイルに対して、ラベル値１が降られていることを確認。

### create train_dataset, val_dataset 

###（3）MakeBalancedDataset Class

In [8]:
from dsets.dsets import MakeBalancedDataset


In [9]:
# 画像の前処理と処理済み画像の表示
# モデルの入力サイズ(タテ・ヨコ)
SIZE = 224
# 標準化する際の各RGBの平均値
MEAN = (0.485, 0.456, 0.406) # ImageNetデータセットの平均値を使用
# 標準化する際の各RGBの標準偏差
STD = (0.229, 0.224, 0.225)  # ImageNetデータセットの標準偏差を使用


# 画像のサイズ、平均値、標準偏差の定数値
size, mean, std = SIZE, MEAN, STD

# MakeDatasetで前処理後の訓練データと正解ラベルを取得
train_dataset = MakeBalancedDataset(
    file_list=file_list, # 訓練データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='train',records=300)
# MakeDatasetで前処理後の検証データと正解ラベルを取得
val_dataset = MakeBalancedDataset(
    file_list=file_list, # 検証データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='val',records=200)


In [10]:
print(len(train_dataset))
print(len(val_dataset))

300
200


###（4）dataloader 

In [11]:
from torch.utils.data import DataLoader

# ミニバッチのサイズを指定
batch_size = 32

# 訓練用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 検証用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [12]:
print(len(train_dl))
print(len(train_dl.dataset))

10
300


In [13]:
print(len(val_dl))
print(len(val_dl.dataset))

7
200


### check

In [14]:
batch_iterator = iter(train_dl)
input_t, label_t = next(batch_iterator)

print(input_t.size())
print(label_t)


torch.Size([32, 3, 224, 224])
tensor([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 1, 0, 1])


In [15]:
batch_iterator = iter(val_dl)
input_t, label_t = next(batch_iterator)

print(input_t.size())
print(label_t.shape)
print(label_t)


torch.Size([32, 3, 224, 224])
torch.Size([32])
tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0])


###（5）For Loop with DataLoader

In [ ]:
import sys
for batch_ndx, batch_tup in enumerate(train_dl):
  input_t, label_t = batch_tup
  print(batch_ndx,input_t.shape)
  #print(label_t)
  var_name = 'input_t'
  print(sys.getsizeof(eval(var_name)))


0 torch.Size([32, 3, 224, 224])
88
1 torch.Size([32, 3, 224, 224])
88
2 torch.Size([32, 3, 224, 224])
88
3 torch.Size([32, 3, 224, 224])
88
4 torch.Size([32, 3, 224, 224])
88
5 torch.Size([32, 3, 224, 224])
88
6 torch.Size([32, 3, 224, 224])
88
7 torch.Size([32, 3, 224, 224])
88
8 torch.Size([32, 3, 224, 224])
88
9 torch.Size([12, 3, 224, 224])
88


In [ ]:
import sys
for batch_ndx, batch_tup in enumerate(val_dl):
  input_t, label_t = batch_tup
  print(batch_ndx,input_t.shape)
  print(label_t)
  var_name = 'input_t'
  print(sys.getsizeof(eval(var_name)))


0 torch.Size([32, 3, 224, 224])
88
1 torch.Size([32, 3, 224, 224])
88
2 torch.Size([32, 3, 224, 224])
88
3 torch.Size([32, 3, 224, 224])
88
4 torch.Size([32, 3, 224, 224])
88
5 torch.Size([32, 3, 224, 224])
88
6 torch.Size([8, 3, 224, 224])
88


#END